In [2]:
import pandas as pd
import numpy as np
import re
!pip install texthero
import texthero as hero
from texthero import preprocessing

     |████████████████████████████████| 235 kB 31.6 MB/s 
     |████████████████████████████████| 1.5 MB 41.0 MB/s 
     |████████████████████████████████| 749 kB 45.2 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
from google.colab import files
uploaded = files.upload()

Saving politifact_scraped_data.csv to politifact_scraped_data.csv


In [8]:
df=pd.read_csv('politifact_scraped_data.csv')
df.head()

,Unnamed: 0,author,statement,source,date,target,BinaryTarget,BinaryNumTarget
0,0,Kiannah Sepeda-Miller,"The migrant caravan from Tapachula, Mexico “is...",Mary Miller,"October 31, 2021",pants-fire,0,FAKE
1,1,Tom Kertscher,Blood “starts to clot in a minute or two” afte...,Instagram posts,"October 29, 2021",false,0,FAKE
2,2,Monique Curet,“Japan drops vax rollout”,Hal Turner Radio Show,"October 29, 2021",pants-fire,0,FAKE
3,3,Samantha Putterman,Virginia voters are being turned away from pol...,Facebook posts,"October 29, 2021",barely-true,0,FAKE
4,4,Ciara O'Rourke,In-N-Out cups say “F--- Joe Biden.”,Viral image,"October 29, 2021",false,0,FAKE


In [9]:
df['statement']=df['statement'].apply(lambda x: clean_html_tags(x))
df['statement'] = hero.clean(df['statement'], custom_pipeline)
df['statement'][0]

'the migrant caravan  tapachula  mexico    population  minneapolis '

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(analyzer='word',ngram_range=(1, 3),min_df=0,stop_words='english')
X_tfidf = tfidf.fit_transform(df['statement'])
X_tfidf

<2400x38942 sparse matrix of type '<class 'numpy.float64'>'
	with 58716 stored elements in Compressed Sparse Row format>

In [12]:
vector = X_tfidf.toarray()
vector[0].shape[0]

38942

In [15]:
!pip install faiss


In [21]:
!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-gpu
import faiss

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (434 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Selecting previously unselected package libomp-dev.
Preparing to unpack .../libomp-dev_5.0.1-1_amd64.deb ...
Unpacking libomp-dev (5.0.1-

# LSH

In [16]:
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors[0].shape[0]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimention,num_bits)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [22]:
%%time
lsh_index = LSHIndex(vector, df['author'])
lsh_index.build()

CPU times: user 206 ms, sys: 130 ms, total: 336 ms
Wall time: 320 ms


In [26]:
query_index = 1
print(f"The most simillar to item {df['author'][query_index]} are:")
items_list = lsh_index.query(np.array([vector[query_index]]).astype('float32'))
df.loc[df['author'].isin(items_list)]

The most simillar to item Tom Kertscher are:


,Unnamed: 0,author,statement,source,date,target,BinaryTarget,BinaryNumTarget
1,1,Tom Kertscher,blood starts clot minute two pfizer co...,Instagram posts,"October 29, 2021",false,0,FAKE
3,3,Samantha Putterman,virginia voters turned away polling locatio...,Facebook posts,"October 29, 2021",barely-true,0,FAKE
4,4,Ciara O'Rourke,in n out cups say f joe biden,Viral image,"October 29, 2021",false,0,FAKE
5,5,Ciara O'Rourke,says cnn headline said in and out fast fo...,Viral image,"October 29, 2021",pants-fire,0,FAKE
7,7,Ciara O'Rourke,trump white house he promises lock hillary,Facebook posts,"October 29, 2021",pants-fire,0,FAKE
...,...,...,...,...,...,...,...,...
2392,2392,Samantha Putterman,says nih years ago published study chlor...,Facebook posts,"July 29, 2020",false,0,FAKE
2393,2393,Warren Fiske,mark warner party bosses calling defund ...,Daniel Gade,"July 29, 2020",false,0,FAKE
2394,2394,Ciara O'Rourke,photo shows huge charter busses black lives ...,Viral image,"July 29, 2020",pants-fire,0,FAKE
2395,2395,Tom Kertscher,walmart pledged million black lives mat...,Facebook posts,"July 28, 2020",false,0,FAKE


# Exhaustive Search

In [27]:
class BruteForceIndex():
    def __init__(self, vectors, labels):
        self.vectors = vectors.astype('float32')
        self.labels = labels
        self.index = faiss.IndexFlatL2(vectors[0].shape[0])
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [29]:
%%time
es_index = BruteForceIndex(vector, df['author'])

CPU times: user 179 ms, sys: 269 ms, total: 448 ms
Wall time: 445 ms


In [31]:
query_index = 1
print(f"The most simillar to item {df['author'][query_index]} are:")
items_list = es_index.query(np.array([vector[query_index]]).astype('float32'))
df.loc[df['author'].isin(items_list)]

The most simillar to item Tom Kertscher are:


,Unnamed: 0,author,statement,source,date,target,BinaryTarget,BinaryNumTarget
1,1,Tom Kertscher,blood starts clot minute two pfizer co...,Instagram posts,"October 29, 2021",false,0,FAKE
3,3,Samantha Putterman,virginia voters turned away polling locatio...,Facebook posts,"October 29, 2021",barely-true,0,FAKE
4,4,Ciara O'Rourke,in n out cups say f joe biden,Viral image,"October 29, 2021",false,0,FAKE
5,5,Ciara O'Rourke,says cnn headline said in and out fast fo...,Viral image,"October 29, 2021",pants-fire,0,FAKE
6,6,Andy Nguyen,atlanta lost police force,Facebook posts,"October 29, 2021",false,0,FAKE
...,...,...,...,...,...,...,...,...
2388,2388,Ciara O'Rourke,hurricane hanna hitting texas blown part ...,Viral image,"July 29, 2020",false,0,FAKE
2392,2392,Samantha Putterman,says nih years ago published study chlor...,Facebook posts,"July 29, 2020",false,0,FAKE
2394,2394,Ciara O'Rourke,photo shows huge charter busses black lives ...,Viral image,"July 29, 2020",pants-fire,0,FAKE
2395,2395,Tom Kertscher,walmart pledged million black lives mat...,Facebook posts,"July 28, 2020",false,0,FAKE


# Product Quantization

In [32]:
class PQIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        m = 3
        n_bits = 8
        self.index = faiss.IndexPQ (self.dimention, m, n_bits)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [33]:
batch_vector = []
for idx, arr in enumerate(vector):
    batch_vector.append(list(np.append(arr, 0.0)))

In [35]:
%%time
batch_vector = np.array(batch_vector)
pq_index = PQIndex(batch_vector, df['author'])
pq_index.build()

CPU times: user 27.2 s, sys: 499 ms, total: 27.7 s
Wall time: 23.5 s


In [39]:
query_index = 1
print(f"The most simillar to item {df['author'][query_index]} are:")
items_list = pq_index.query(np.array([batch_vector[query_index]]).astype('float32'))
df.loc[df['author'].isin(items_list)]

The most simillar to item Tom Kertscher are:


,Unnamed: 0,author,statement,source,date,target,BinaryTarget,BinaryNumTarget
1,1,Tom Kertscher,blood starts clot minute two pfizer co...,Instagram posts,"October 29, 2021",false,0,FAKE
3,3,Samantha Putterman,virginia voters turned away polling locatio...,Facebook posts,"October 29, 2021",barely-true,0,FAKE
6,6,Andy Nguyen,atlanta lost police force,Facebook posts,"October 29, 2021",false,0,FAKE
9,9,Tom Kertscher,there thing booster there failed prod...,Viral image,"October 29, 2021",false,0,FAKE
13,13,Tom Kertscher,in united kingdom plus percent people ...,Alex Berenson,"October 29, 2021",false,0,FAKE
...,...,...,...,...,...,...,...,...
2380,2380,Samantha Putterman,the longest filibuster u s history days ...,Facebook posts,"July 30, 2020",barely-true,0,FAKE
2386,2386,Tom Kertscher,american coronavirus vaccine killed five ukra...,Facebook posts,"July 30, 2020",false,0,FAKE
2392,2392,Samantha Putterman,says nih years ago published study chlor...,Facebook posts,"July 29, 2020",false,0,FAKE
2395,2395,Tom Kertscher,walmart pledged million black lives mat...,Facebook posts,"July 28, 2020",false,0,FAKE


# Trees and Graphs

In [40]:
!pip install annoy
import annoy

     |████████████████████████████████| 646 kB 31.4 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391674 sha256=cc1635489a7ebb496109c15e31409ae67da87315d37080ade539bd4f15decffa
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [1]:
!pip install apache_beam
!pip install 'scikit_learn~=0.23.0'  # For gaussian_random_matrix.
!pip install annoy

In [ ]:
%%time
tg_index = TreeGraphIndex(vector, df['author'])
tg_index.build()

In [ ]:
query_index = 1
print(f"The most simillar to item {df['author'][query_index]} are:")
items_list = tg_index.query(vector[query_index])
df.loc[df['author'].isin(items_list)]

# ierarchical Navigable Small Worlds (HNSW)

In [6]:
!pip install pip==9.0.3 pybind11
!pip install nmslib
import nmslib

    100% |████████████████████████████████| 8.6MB 135kB/s 
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 194kB 5.6MB/s 
  Found existing installation: pybind11 2.8.1
    Uninstalling pybind11-2.8.1:
      Successfully uninstalled pybind11-2.8.1


In [7]:
class HNSWIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors[0].shape[0]
        self.vectors = vectors.astype('float32')
        self.labels = labels

    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

In [ ]:
%%time
hnsw_index = HNSWIndex(vector, df['author'])
hnsw_index.build()

Sum

# Sum

LSH:CPU times: 
user 206 ms, sys: 130 ms, total: 336 ms
Wall time: 320 ms

Exhaustive Search:
CPU times: user 179 ms, sys: 269 ms, total: 448 ms
Wall time: 445 ms

Product Quantization:
CPU times: user 27.2 s, sys: 499 ms, total: 27.7 s
Wall time: 23.5 s

Trees and Graphs:

HNSW: